<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week04_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 4



## Shallow Neural Networks



### 7.1 Neural Networks in One Dimension



In [ ]:
import os


import matplotlib.pyplot as plt
import torch
import torch.nn as nn



In [ ]:
torch.manual_seed(0)


class Net(torch.nn.Module):
  def __init__(self, *argv, **kwarg):
    super(Net, self).__init__()

    self.linears = torch.nn.ModuleList([])
    for n_in, n_out in (argv[:-1], argv[1:]):
      self.linears.append(torch.nn.Linear(n_in, n_out))

  def forward(self, x):

    for layer in self.linears:
      x = torch.sigmoid(layer(x))

    return x

  def plot_activation(self, Y, X):
      a1 = torch.sigmoid(self.linears[0](X))
      plt.scatter(
          a1.detach().numpy()[:, 0],
          a1.detach().numpy()[:, 1],
          c=Y.numpy().reshape(-1)
      )
      plt.title('activations')
      plt.grid(True)


# https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html
# https://stackoverflow.com/questions/50463975/pytorch-how-to-properly-create-a-list-of-nn-linear
# https://discuss.pytorch.org/t/when-should-i-use-nn-modulelist-and-when-should-i-use-nn-sequential/5463



* 1 input node
* two nodes in the input layer
* one node on the output layer



$$
\begin{align}
z_{N \times 2}^1&=x_{N \times 1}^1W_{1 \times 2}^1+b_{N \times 2}^1 \\
x_{N \times 2}^2&=\sigma(z_{N \times 2}^1) \\
z_{N \times 1}^2&=x_{N \times 2}^2W_{2 \times 1}^2+b_{N \times 1}^2 \\
y_{N \times 1}&=\sigma(z_{N \times 1}^2)
\end{align}
$$



In [ ]:
model = Net(1, 2, 1)

x = torch.tensor([0.0])

yhat = model(x)
yhat



In [ ]:
model.state_dict()



Sequential version



In [ ]:
torch.manual_seed(1)

model = torch.nn.Sequential(
    torch.nn.Linear(1, 2),
    torch.nn.Sigmoid(),
    torch.nn.Linear(2, 1),
    torch.nn.Sigmoid(),
)

x = torch.tensor([0.0])

yhat = model(x)

yhat



In [ ]:
x = torch.tensor([
    [0.0],
    [1.0],
    [2.0],
  ])

yhat = model(x)

yhat



In [ ]:
model.state_dict()



Train the model



In [ ]:
X = torch.linspace(-20, 20, 41).view(-1, 1).type(torch.FloatTensor)
Y = torch.zeros(X.shape[0])
Y[
    (X[:, 0] > -4.0) & (X[:, 0] < 4.0)
] = 1.0
Y = Y.view(-1, 1)



In [ ]:
def plot_y_yhat(X, Y, model, epoch, legend=True):
    plt.plot(
        X.numpy(), model(X).detach().numpy(),
        label=('epoch '+str(epoch))
    )
    plt.plot(X.numpy(), Y.numpy(), 'r')
    plt.xlabel('x')
    if legend:
      plt.legend(loc=0)
    plt.grid(True)



In [ ]:
def train(Y, X, model, optimizer, criterion, n_epoch=1000):
  cost = []
  total = 0

  if os.environ.get('CI', False):
    n_epoch = 1

  for epoch in range(n_epoch):
    total = 0.0

    for y, x in zip(Y, X):
      yhat = model(x)

      # https://stackoverflow.com/questions/57798033/valueerror-target-size-torch-size16-must-be-the-same-as-input-size-torch
      loss = criterion(yhat, y)
      loss.backward()

      optimizer.step()
      optimizer.zero_grad()

      total += loss.item()

    cost.append(total)

  return cost



In [ ]:
model = Net(1, 2, 1)

plot_y_yhat(X, Y, model, 0, legend=True)



In [ ]:
model.plot_activation(Y, X)



In [ ]:
loss_list = train(
    Y=Y, X=X, model=model,
    optimizer=torch.optim.SGD(model.parameters(), lr=0.1),
    criterion=torch.nn.BCELoss(),
    n_epoch=1 # use 1000 to train
)



In [ ]:
if len(loss_list) > 10:
  plot_y_yhat(X, Y, model, len(loss_list), legend=True)



In [ ]:
if len(loss_list) > 10:
  model.plot_activation(Y, X)



In [ ]:
if len(loss_list) > 10:
  plt.plot(loss_list)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.grid(True)



### 7.2 Neural Networks More Hidden Neurons



More complicated data



In [ ]:
X = torch.linspace(-20, 20, 41).view(-1, 1).type(torch.FloatTensor)
Y = torch.zeros(X.shape[0])
Y[
    ((X[:, 0] > -10.0) & (X[:, 0] < -5.0))
    | ((X[:, 0] > 5.0) & (X[:, 0] < 10.0))
] = 1.0
Y = Y.view(-1, 1)



How would the previous model do?



In [ ]:
model = Net(1, 2, 1)

plot_y_yhat(X, Y, model, 0, legend=True)



In [ ]:
model.plot_activation(Y, X)



In [ ]:
loss_list = train(
    Y=Y, X=X, model=model,
    optimizer=torch.optim.SGD(model.parameters(), lr=0.1),
    criterion=torch.nn.BCELoss(),
    n_epoch=1, # use 1000 to train
)



In [ ]:
if len(loss_list) > 10:
  plot_y_yhat(X, Y, model, len(loss_list), legend=True)



In [ ]:
if len(loss_list) > 10:
  model.plot_activation(Y, X)



In [ ]:
if len(loss_list) > 10:
  plt.plot(loss_list)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.grid(True)



Add more neurons to the hidden layer



In [ ]:
torch.manual_seed(0)
model = Net(1, 9, 1)



Define the dataset


In [ ]:
class Data(torch.utils.data.Dataset):
  def __init__(self):
    self.x = torch.linspace(-20, 20, 100).view(-1, 1).type(torch.FloatTensor)

    self.y = torch.zeros(self.x.shape[0])
    self.y[
        ((self.x[:, 0] > -10.0) & (self.x[:, 0] < -5.0))
        | ((self.x[:, 0] > 5.0) & (self.x[:, 0] < 10.0))
    ] = 1.0

    self.y = self.y.view(-1, 1)
    self.len = self.x.shape[0]

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.len

  def plot_y_yhat(self, model, epoch, legend=True):
    plot_y_yhat(self.x, self.y, model, epoch, legend=True)



In [ ]:
data_set = Data()
data_set.plot_y_yhat(model, 0, legend=True)



In [ ]:
model.plot_activation(Y, X)



train() function with `trainloader`



In [ ]:
def train(model, optimizer, criterion, train_loader, n_epoch=1000):
  cost = []

  if os.environ.get('CI', False):
    n_epoch = 1

  for epoch in range(n_epoch):
    total = 0.0

    for x, y in train_loader:
      optimizer.zero_grad()
      yhat = model(x)

      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()

      optimizer.step()

      total += loss.item()

    cost.append(total)

  return cost



In [ ]:
%%time
loss_list = train(
    model=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=0.01),
    criterion=torch.nn.BCELoss(),
    train_loader=torch.utils.data.DataLoader(dataset=data_set, batch_size=100),
    n_epoch=1 # use 1000 to train
)



In [ ]:
if len(loss_list) > 10:
  plot_y_yhat(X, Y, model, len(loss_list), legend=True)



In [ ]:
if len(loss_list) > 10:
  plt.plot(loss_list)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.grid(True)



### 7.3 Neural Networks with Multiple Dimensional Input



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mc


import torch
import torch.nn
import torch.utils.data



In [ ]:
class XOR_Data(torch.utils.data.Dataset):

  def __init__(self, N_s=100, noise=0.1):

    self.x = torch.zeros((N_s, 2))
    self.y = torch.zeros((N_s, 1))

    Ns_4 = N_s // 4

    for i in range(0, Ns_4):
      self.x[i, :] = torch.Tensor([0.0, 0.0])
      self.y[i, 0] = torch.Tensor([0.0])

    for i in range(Ns_4, 2 * Ns_4):
      self.x[i, :] = torch.Tensor([0.0, 1.0])
      self.y[i, 0] = torch.Tensor([1.0])

    for i in range(2 * Ns_4, 3 * Ns_4):
      self.x[i, :] = torch.Tensor([1.0, 0.0])
      self.y[i, 0] = torch.Tensor([1.0])

    for i in range(3 * Ns_4, N_s):
      self.x[i, :] = torch.Tensor([1.0, 1.0])
      self.y[i, 0] = torch.Tensor([0.0])

    self.x += noise * torch.randn((N_s, 2))
    self.len = N_s

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.len

  def plot(self):
    _, ax = plt.subplots(subplot_kw={"projection": "3d"})

    ax.plot(
      self.x[:, 0].numpy(),
      self.x[:, 1].numpy(),
      self.y[:, 0].numpy(),
      '.',
    )




In [ ]:
data_set = XOR_Data()



Plot of data



In [ ]:
data_set.plot()



Model



In [ ]:
model = Net(2, 4, 1)



In [ ]:
%%time
loss_list = train(
    model=model,
    optimizer=torch.optim.SGD(model.parameters(), lr=0.01),
    criterion=torch.nn.BCELoss(),
    train_loader=torch.utils.data.DataLoader(dataset=data_set, batch_size=1),
    n_epoch=500 # use 500 to train
)



In [ ]:
if len(loss_list) > 10:
  plt.plot(loss_list)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.grid(True)



### 7.4 Multi-Class Neural Networks



### 7.5 Backpropagation



### 7.6 Activation Function

